<a href="https://colab.research.google.com/github/gabrielbauer/T1_ML/blob/main/Trabalho_Aprendizado_de_M%C3%A1quina.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Componentes do grupo:
- Gabriel Bauer de Oliveira - 00275614
- Vitorugu da Silveira Fuerstenau Maciel - 007
- Fábio Petkowicz - 

Base de dados: https://www.kaggle.com/datasets/clmentbisaillon/fake-and-real-news-dataset?resource=download&select=True.csv

##Inspiration
Can you use this data set to make an algorithm able to determine if an article is fake news or not?

In [ ]:
!unzip "/content/archive.zip" -d "/content/true_and_false"

Archive:  /content/archive.zip
replace /content/true_and_false/Fake.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: N
/content


In [45]:
import pandas as pd
from IPython.display import display

true_news_dataframe = pd.read_csv("true_and_false/True.csv")
fake_news_dataframe = pd.read_csv("true_and_false/Fake.csv")

list(fake_news_dataframe.columns)

['title', 'text', 'subject', 'date']

Verificação de colunas com campos vazios

In [67]:
colunas = ['title', 'text', 'subject', 'date']
dataframes = [true_news_dataframe, fake_news_dataframe]

valido = 1
for dataframe in dataframes:
  for coluna in colunas:
   if (dataframe.loc[dataframe[coluna] == None].empty == False):
      valido = -1
      break

print("Dataframes sem colunas vazias.") if valido == 1 else print("Dataframe com colunas vazias.")


Dataframes sem colunas vazias.


Realizar uma implementação própria da estratégia de k-fold cross validation, que seja genérica o suficiente para permitir o uso de diferentes valores de k (número de folds). A estratégia deve ser aplicada de forma estratificada, isto é, mantendo-se a proporção original de exemplos por classe em todos os folds, independente do número de classes do problema. É aconselhável que os grupos estudem as funções prontas a serem utilizadas para treinamento dos modelos, a fim de implementarem sua função para validação cruzada de forma compatível com as mesmas (principalmente em relação a estruturas de dados).